<a href="https://colab.research.google.com/github/juliawol/WB_Knowledge_Base/blob/main/WB_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymorphy2-dicts
!pip install pymorphy2
!pip install --user -U nltk


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer

# Initialize tools for preprocessing
morph = MorphAnalyzer()
stop_words = stopwords.words('russian')

# Define terminology and create a vocabulary list from it
terminology = [
    ("ТП", "торговая площадка. Платформа Wildberries."),
    ("ПВЗ", "пункт выдачи заказов. Место, куда покупатели приходят за посылками."),
    ("ШК офиса", "уникальный штрихкод, который даёт доступ к рабочему интерфейсу NPOS."),
    ("ID офиса", "номер пункта выдачи в системе Wildberries."),
    ("ID менеджера", "номер учетной записи в системе Wildberries."),
    ("ШК", "штрихкод. На упаковке каждого товара и на приходных коробках."),
    ("Стикер", "помогает узнать информацию о заказе, но не используется для поиска товара."),
    ("Баркод", "штрихкод производителя. Используется для сверки данных о товаре."),
    ("QR-код", "двумерный штрихкод с информацией, расшифровывается сканером."),
    ("Волна или волнорез", "стеллаж, где хранятся товары."),
    ("Приходная коробка", "упаковка, в которой заказы приходят в пункт выдачи."),
    ("Невостребованный товар", "товар, который покупатель не забрал из ПВЗ в течение 12 дней."),
    ("Невозвратный товар", "товар, который нельзя вернуть."),
    ("Возвратная коробка", "упаковка, в которой невостребованные товары отправляют обратно на склад."),
    ("Возвратная наклейка", "элемент упаковки со штрихкодом и номером коробки."),
    ("Сейф-пакет", "специальная упаковка для ювелирных изделий и гаджетов.")
]
terminology_dict = dict(terminology)
custom_vocabulary = list(terminology_dict.keys())

# Load data
train_data_df = pd.read_csv("/content/train_data.csv")
chunks_df = pd.read_csv("/content/chunks.csv")

# Retrieve the original chunks for final output
chunk_texts_original = chunks_df['Chunk'].tolist()

# Function to preprocess text: punctuation removal, lemmatization, stop-word filtering
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize and lemmatize, removing stop words
    tokens = word_tokenize(text)
    tokens = [morph.parse(token)[0].normal_form for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# Preprocess chunks and extend each chunk with relevant terminology definitions if applicable
chunk_texts_preprocessed = []
for chunk in chunk_texts_original:
    # Handle missing values (NaN) by replacing them with an empty string
    if pd.isnull(chunk):  # Check if chunk is NaN
        chunk = ""  # Replace NaN with empty string
    # Preprocess the chunk first
    preprocessed_chunk = preprocess_text(chunk)
    # Check for terms in the preprocessed chunk
    terms_in_chunk = [term for term in custom_vocabulary if term.lower() in preprocessed_chunk]
    # Append definitions of found terms
    definitions = " ".join(terminology_dict.get(term, '') for term in terms_in_chunk)
    # Combine and preprocess again
    combined_text = preprocess_text(chunk + " " + definitions)
    chunk_texts_preprocessed.append(combined_text)


# Vectorize chunks using TF-IDF without custom vocabulary
vectorizer = TfidfVectorizer()
chunk_vectors = vectorizer.fit_transform(chunk_texts_preprocessed)

# Function to find top-k relevant chunks for a question, retrieving original unprocessed chunks
def find_relevant_chunks_tfidf(question, top_k=5):
    # Preprocess the question for similarity search
    question_preprocessed = preprocess_text(question)
    question_vector = vectorizer.transform([question_preprocessed])

    # Debugging: Check if the question vector is not all zeros
    if not question_vector.nnz:
        print("Warning: Question vector is all zeros.")

    # Calculate cosine similarities and retrieve top-k indices
    cosine_similarities = cosine_similarity(question_vector, chunk_vectors).flatten()
    top_indices = cosine_similarities.argsort()[-top_k:][::-1]

    # Retrieve original, unprocessed chunks for readability
    relevant_chunks = [chunk_texts_original[i] for i in top_indices]
    return relevant_chunks

# Example usage
question = "Как завершить приемку товара без штрихкода?"
relevant_chunks = find_relevant_chunks_tfidf(question)
print("/nTF-IDF Relevant Chunks:", relevant_chunks)


/nTF-IDF Relevant Chunks: ['На товаре нет штрихкода при приемке: Способ 1  1. Если на товаре есть баркод, откройте раздел «Движение вещей» 2. Пикните баркод вещи — в программе отобразится приходная коробка и штрихкод товара  Способ 2  1. Запишите данные коробки, в которой был товар 2. Когда завершите приёмку, создайте обращение 3. Укажите номер коробки и баркод вещи — КЦ пришлют правильный штрихкод  Способ 3  1. После того как приняли и разложили все товары, откройте раздел «Движение вещей» 2. Выберите нужную дату: в окне отобразится операция «Завершение приёмки коробки» 3. Скопируйте номер из столбца «ШК / Стикер» 4. Перейдите в раздел «История ШК» и вставьте номер в пустое поле 5. Сравните вещь с карточкой товара. Если совпадает, примите товар вручную и напишите ШК на пакете. Если нет, напишите руководителю или в поддержку', 'Если на товаре при приемке вместо ШК стикер с QR-кодом, сканируйте его так же, как и обычный штрихкод.', 'Одновременно можно принимать до 10 коробок. Вы также м

In [ ]:
# Example usage
question = "Кто решает, когда делать генеральную уборку?"
relevant_chunks = find_relevant_chunks_tfidf(question)
print("TF-IDF Relevant Chunks:", relevant_chunks)

TF-IDF Relevant Chunks: ['Как часто проводить генеральную уборку, решает руководитель пункта. Обычно — от 1 раза в неделю до 1 раза в 1-2 месяца.  Что сделать:  - Выполнить все пункты из чек-листа «Ежедневная уборка» - Убрать паутину из углов - Отмыть ножи и ножницы от скотча, заменить лезвия - Протереть технику, антивандальный ящик - Протереть мебель - Протереть огнетушители и подставки под них - Постирать шторы из примерочных - Помыть коврики - Заказать расходники, если что-то закончилось', 'Mikrotik роутеры настраивают руководители филиалов. Менеджерам не нужно ничего делать', 'Приёмка товаров делится на несколько этапов: курьер привозит заказы в приходных коробках, менеджер сверяет адрес и принимает коробки на баланс, а потом распределяет товары по ячейкам. Рассказываем подробнее, что делать на каждом из этапов.', 'Какие расходные материалы должны быть в пункте: - Средство для мытья пола, - Средство для мытья стёкол, - Скотч, - Канцелярские ножи и сменные лезвия, - Тряпки для уборк

In [ ]:
# Example usage
question = "Что делать, если в заказе есть товар, который нельзя вернуть?"
relevant_chunks = find_relevant_chunks_tfidf(question)
print("TF-IDF Relevant Chunks:", relevant_chunks)

TF-IDF Relevant Chunks: ['Конфликты и недовольство чаще всего возникают, если: - менеджер делает что-то не так, например, случайно выдаёт невозвратный товар; - покупатель невнимательно оформляет заказ, например, не замечает, что отказ от товара платный или вещь невозвратная.  Рассказываем, как вести себя в конфликтной ситуации, в инструкции Как построить конструктивный диалог с покупателем\u200b', 'Какими бывают статусы заказа: -Оформлен: заказ принят в системе - Отправлен на сборку: товар проверяют и готовят к отправке - Собран: товар в наличии, заказ подтвердили и упаковали - Отсортирован: заказ готов к транспортировке - В пути на СЦ: заказ доставляют в сортировочный центр - В пути на пункт выдачи: заказ везут в пункт выдачи - Готов к выдаче: заказ можно забирать - Получен: покупатель забрал заказ - Возврат: покупатель вернул товар', 'Если в заказе несколько позиций, вернуть на полку можно только все товары вместе — разделять заказ нельзя. Чтобы отложить заказ, нажмите кнопку «Снять 

In [ ]:
# Example usage
question = "Что делать, сломалась вывеска?"
relevant_chunks = find_relevant_chunks_tfidf(question)
print("TF-IDF Relevant Chunks:", relevant_chunks)

TF-IDF Relevant Chunks: ['Вывеска не работает, зеркало треснуло, стеллаж упал, у лавочки сломалась ножка? Сообщите об этом руководителю, чтобы он помог с ремонтом или заменой', 'Если сломалась или повредилась мебель, зеркало или коврик, сообщите руководителю, что нужен ремонт. - Если сломался светильник или перегорела лампочка, замените на такие же самостоятельно или сообщите руководителю о проблеме. - Если сломался тейбл-тент — подставка для объявлений — закажите новый на WB или попросите руководителя. Артикулы: 20868971 или 97295309  Нельзя использовать картон вместо тейбл-тентов и писать объявления от руки', 'Если сломалась вывеска Wildberries, лайтбокс, лестница, перила, пандус, дверь, дверная ручка или доводчик, сообщите руководителю, что нужен ремонт. - Если нужно заменить наклейку с режимом работы, закажите новую на WB. Найти наклейку можно по запросу: «Режимник Wildberries наклейка». Перед заказом проверьте, что время на наклейке совпадает с режимом работы вашего пункта. - Если

In [ ]:
# Example usage
question = "Что делать, сломался роутер"
relevant_chunks = find_relevant_chunks_tfidf(question)
print("TF-IDF Relevant Chunks:", relevant_chunks)

TF-IDF Relevant Chunks: ['Mikrotik роутеры настраивают руководители филиалов. Менеджерам не нужно ничего делать', 'Если сломалась или повредилась мебель, зеркало или коврик, сообщите руководителю, что нужен ремонт. - Если сломался светильник или перегорела лампочка, замените на такие же самостоятельно или сообщите руководителю о проблеме. - Если сломался тейбл-тент — подставка для объявлений — закажите новый на WB или попросите руководителя. Артикулы: 20868971 или 97295309  Нельзя использовать картон вместо тейбл-тентов и писать объявления от руки', 'Настройка роутера ASUS  https://docs.google.com/document/d/1LrPu34EVSozrJ6suZLTwcQTgZgqULttM34v04yFhhr0/edit#heading=h.drjxa7ls1f3y', 'Настройка роутера HUAWEI  https://docs.google.com/document/d/1QvreMLX9jdo2e0ea_XSMPzfpFPmncUVHA_BPvV475mg/edit#heading=h.trgtokopugrp', 'Настройка роутера TP-Link  https://docs.google.com/document/d/1MZ1--LGUhcu6CwGtiS9j2XD9_n88luvXTLgGW2EFaqg/edit']


In [ ]:
# Define a function to calculate Recall@k for TF-IDF model
def calculate_recall_at_k(test_data, top_k_values=[1, 3, 5]):
    recall_scores = {k: 0 for k in top_k_values}
    total_questions = len(test_data)

    for _, row in test_data.iterrows():
        question = row['Question']
        true_chunk = row['Chunk']

        retrieved_chunks = find_relevant_chunks_tfidf(question, top_k=max(top_k_values))

        for k in top_k_values:
            if true_chunk in retrieved_chunks[:k]:
                recall_scores[k] += 1

    recall_at_k = {f"Recall@{k}": recall_scores[k] / total_questions for k in top_k_values}
    return recall_at_k

# Test the recall calculation on a sample of the test data
sampled_test_data = train_data_df.sample(frac=0.1, random_state=42)  # Sample 10% for testing
recall_scores = calculate_recall_at_k(sampled_test_data, top_k_values=[1, 3, 5])

# Display recall scores
print(recall_scores)

{'Recall@1': 0.25, 'Recall@3': 0.5, 'Recall@5': 0.5833333333333334}
